# Gaussian Process

In [1]:
import pandas as pd
import plotly.graph_objects as go
import math
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def generate_function(x=np.linspace(start=0, stop=100, num=1000), A=[1], c=[0], k=[1], b=[0], active=[1], dead_schedule=[None], theme=None, title=None):
  
  # init
  dt       = pd.DataFrame()
  convolve = []
  fig      = go.Figure()



  for i in zip(A,c,k,b,active,dead_schedule):
    # ---- compute sin ----x
    sn       = np.array([i[0]*math.sin(i[2]*xx-i[1]) + i[3] for xx in x])
    
    # ---- partically inactive ----x
    if i[4] == 0:
      index     = np.random.choice(np.linspace(1,100-i[5][1]), i[5][0]).round()
      indexing  = np.repeat(0, len(x))
      end_index = [j + i[5][1] for j in index]

      for id in range(len(index)):
        mask           = ((x >= index[id]) & (x <= end_index[id]))
        indexing[mask] = 1
      sn = indexing * sn

  
    name = 'A:' + str(i[0]) + '  c:' + str(i[1]) + '  k:' + str(i[2]) + '  b:' + str(i[3]) + ' act:' + str(i[4])
    data = {'name': np.repeat(name, len(x)), 'x':x, 'y':sn}
    
    dt = dt.append(pd.DataFrame(data))
    convolve.append(sn)
    fig.add_trace(go.Scatter(x=x, y=sn, mode='lines', name=name, line=dict(width=1, dash='dot')))

  # ---- convolution ----x
  conv = convolve[0]
  for c in convolve[1:]:
    conv = conv + c

  data = {'name': np.repeat('convolution', len(x)), 'x': x, 'y': conv}
  dt   = dt.append(pd.DataFrame(data))

  

  fig.add_trace(go.Scatter(x=x, y=conv, mode='lines', name='convolution', line=dict(color='steelblue', width=2.5)))
  fig.update_layout(template=theme, title=title, showlegend=False, hovermode=False)




  fig.show()

  return(dt)

In [3]:

A=[1,5,2,1]
c=[0,5,-3,2]
k=[1,.1,0.25,0.4]
b=[0,0,-0,1]
active=[1,1,1,0]

# n activations: of size k
dead_schedule = [
  # [n k]
  [None], [None], [None],
  [3,20]
]


data = generate_function(A=A, c=c, k=k, b=b, active=active, dead_schedule=dead_schedule, 
                         theme='plotly_white', title='Sinusoidal Functional Analysis')

In [4]:
import plotly.io as pio
pio.templates
target = data.loc[data['name']=='convolution', ]

In [5]:
# --- generate random data ----x
rand = np.random.choice(range(len(target)), 1000, replace=True)

sample = target.loc[rand,:]
sample.loc[:,'y'] = sample.loc[:,'y'] + np.random.normal(0, 1, size=len(target))

In [6]:
fig      = go.Figure()

fig.add_trace(go.Scatter(x=sample.loc[:,'x'], y=sample.loc[:,'y'], mode='markers', name='Samples',
marker=dict(
            color='MediumPurple',
            size=1.5,
            line=dict(
                color='MediumPurple',
                width=1.5
            ))))





fig.add_trace(go.Scatter(x=target.loc[:,'x'], y=target.loc[:,'y'], mode='lines', name='Target Function', 
line=dict(color='LightSkyBlue', width=2)))
fig.update_layout(template=None, title='Sample Distribution', showlegend=False, hovermode=False)
fig.show()

In [21]:
# load pystan
import pystan
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import scipy.stats as stats


In [27]:
n    = 100
mu   = 10
sd   = 2
data = np.random.normal(mu, sd, n)
	
data[0:5]

array([11.04709204, 11.64303462,  9.7025163 ,  9.90925918,  8.59101415])

In [34]:
norm_code = """
data {
    int<lower=0> n;
    real y[n];
}
parameters {
    real<lower=0, upper=100> mu;
    real<lower=0, upper=10> sigma;
}
model {
    y ~ normal(mu, sigma);
}
"""

In [35]:
model = pystan.StanModel(model_code=norm_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_01c78a21aa51883a9aabd4563199e4c5 NOW.


LinkError: command 'x86_64-apple-darwin13.4.0-clang++' failed with exit status 1

In [ ]:
norm_data = {
             'n': n,
             'y': data,
            }

In [ ]:
coin_dict = fit.extract()
coin_dict.keys()
# lp_ is the log posterior

In [37]:


fig      = go.Figure()

fig.add_trace()
fig.add_trace(go.Scatter(x=target.loc[:,'x'], y=target.loc[:,'y'], mode='lines', name='Target Function', 
line=dict(color='#7dc4f0', width=2.5, dash='dot')))
fig.update_layout(template=None, title='Target Function', showlegend=False, hovermode=False)
fig.show()

TypeError: add_trace() missing 1 required positional argument: 'trace'

In [80]:
!conda install -c conda-forge pystan

Solving environment: done

# All requested packages already installed.



In [ ]:
# import stan

# schools_code = """
# data {
#   int<lower=0> J;         // number of schools
#   real y[J];              // estimated treatment effects
#   real<lower=0> sigma[J]; // standard error of effect estimates
# }
# parameters {
#   real mu;                // population treatment effect
#   real<lower=0> tau;      // standard deviation in treatment effects
#   vector[J] eta;          // unscaled deviation from mu by school
# }
# transformed parameters {
#   vector[J] theta = mu + tau * eta;        // school treatment effects
# }
# model {
#   target += normal_lpdf(eta | 0, 1);       // prior log-density
#   target += normal_lpdf(y | theta, sigma); // log-likelihood
# }
# """

# schools_data = {"J": 8,
#                 "y": [28,  8, -3,  7, -1,  1, 18, 12],
#                 "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

# posterior = stan.build(schools_code, data=schools_data)
# fit = posterior.sample(num_chains=4, num_samples=1000)
# eta = fit["eta"]  # array with shape (8, 4000)
# df = fit.to_frame()  # pandas `DataFrame, requires pandas

In [98]:
import stan

ModuleNotFoundError: No module named 'stan'